<h1 id="Practical-Data-Science-2016,-Assignment-3">Practical Data Science 2016, Assignment 3<a class="anchor-link" href="#Practical-Data-Science-2016,-Assignment-3">&#182;</a></h1><p>In this assignment,we participate in a <a href="https://www.kaggle.com/">Kaggle</a> competition and specifically  <a href="https://www.kaggle.com/c/transfer-learning-on-stack-exchange-tags">Transfer Learning on Stack Exchange Tags</a>.</p>
<hr>

<blockquote><p>Dimitrios Karamanis <br />
Department of Informatics <br />
Athens University of Economics and Business <br />
dkaramanis@aueb.gr</p>
</blockquote>

<h2 id="Question">Question<a class="anchor-link" href="#Question">&#182;</a></h2><p>Provide a solution to the competition, in the best way you can. Some explanations on what the competition is about:</p>
<ul>
<li><p>You are given a set of Stack Exchange data dumps, in six different domains:</p>
<ul>
<li>biology</li>
<li>cooking</li>
<li>cryptography</li>
<li>DIY</li>
<li>robotics</li>
<li>travel. </li>
</ul>
<p>Each data dump contains question titles, content, and tags.</p>
</li>
<li><p>Your goal is to find a way to predict <em>tags</em>. But your way must be subject-neutral.</p>
</li>
<li><p>So, to test how well your approach works, you will use as your test data a dump from an entirely different domain: physics. You should find a method, based on the dumps of the six domains, that you can use to find tags in physics.</p>

<h2 id="Question-1">Reading the data<a class="anchor-link" href="#Question-1">&#182;</a></h2>
<p>We downloal the data from the competition's site and read them after importing necessary libraries</p>

In [144]:
#import libraries
import pandas as pd
import sklearn as sk
import numpy as np
import string
import re
import gensim
from gensim.parsing.preprocessing import STOPWORDS
import stop_words
from gensim.corpora.dictionary import Dictionary
from sklearn import svm
from gensim import corpora, models, similarities
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from nltk.probability import FreqDist
import distance
import RAKE
import operator


In [145]:
#read the data
df1 = pd.read_csv('biology.csv')
df2 = pd.read_csv('cooking.csv')
df3 = pd.read_csv('crypto.csv')
df4 = pd.read_csv('diy.csv')
df5 = pd.read_csv('robotics.csv')
df6 = pd.read_csv('travel.csv')
df7 = pd.read_csv('test.csv')

#set category columns
df1['category']='biology'
df2['category']='cooking'
df3['category']='crypto'
df4['category']='diy'
df5['category']='robotics'
df6['category']='travel'
df7['category']='physics'
df7['tags']=""

In [146]:
#concatenate the data and put category number
all_df = pd.concat([ df[['title','content','tags','category']] for df in [df1, df2, df3, df4, df5,df6] ])
all_df['category_num'] = all_df.category.map({'biology':0, 'cooking':1,'crypto':2, 'diy':3,'robotics':4, 'travel':5})

<p>Let's see how the data look:</p>

In [147]:
all_df.head(5)

,title,content,tags,category,category_num
0,What is the criticality of the ribosome bindin...,"<p>In prokaryotic translation, how critical fo...",ribosome binding-sites translation synthetic-b...,biology,0
1,How is RNAse contamination in RNA based experi...,<p>Does anyone have any suggestions to prevent...,rna biochemistry,biology,0
2,Are lymphocyte sizes clustered in two groups?,<p>Tortora writes in <em>Principles of Anatomy...,immunology cell-biology hematology,biology,0
3,How long does antibiotic-dosed LB maintain goo...,<p>Various people in our lab will prepare a li...,cell-culture,biology,0
4,Is exon order always preserved in splicing?,<p>Are there any cases in which the splicing m...,splicing mrna spliceosome introns exons,biology,0


<p>and the data for which we want to predict tags </p>

In [148]:
df7.head(5)

,id,title,content,category,tags
0,1,What is spin as it relates to subatomic partic...,<p>I often hear about subatomic particles havi...,physics,
1,2,What is your simplest explanation of the strin...,<p>How would you explain string theory to non ...,physics,
2,3,"Lie theory, Representations and particle physics",<p>This is a question that has been posted at ...,physics,
3,7,Will Determinism be ever possible?,<p>What are the main problems that we need to ...,physics,
4,9,Hamilton's Principle,<p>Hamilton's principle states that a dynamic ...,physics,


<p>Now we will set stopwords that were imported from libraries and add additional one's from a freely available stopword list generated by Chris Buckley and Gerard Salton at Cornell University plus several words that we added after several tries  <a href="http://www.lextek.com/manuals/onix/stopwords2.html ">http://www.lextek.com/manuals/onix/stopwords2.html </a> </p>


In [149]:
additional_stopwords = set(["nofollow","sin", "pi","dt" ,"ray","red","20","black","vec","difference","wiki", "rest","charged", "frac","gt","nu","psi","work","abs","png","omega","change","vec","day","set","high","stackexchange" , "hat","rod","int","mu", "read","time", "image", "people", "case","line", "years","problem", "description", "imgur", "code","key", "img", "amp","emage", "problem", "good", "bit", "questions","hot", "room","message", "random","run", "hash", "understand","add", "number",  "blockquote","stack","question","run", "enter","light","long","alt","days","small", "has","long",'www',"end", 'http','href','https', 'com', 'org', 'x', 'li','0', '1', '2', '3','4', '5', "a","br","i","ol", "tr","u","td","li","il","em","rel", 
'6', "lt","ve", "src", "strong","ul","10", "en", "pre",'7','8', '9','t','s', 'm', 'p', 'n',"a","a's","able","about","above","according","accordingly","across","actually","after","afterwards",
"again","against","ain't","all","allow","allows","almost","alone","along","already","also","although",
"always","am","among","amongst","an","and","another","any","anybody","anyhow","anyone","anything","anyway",
"anyways","anywhere","apart","appear","appreciate","appropriate","are","aren't","around","as","aside","ask",
"asking","associated","at","available","away","awfully","b","be","became","because","become","becomes",
"becoming","been","before","beforehand","behind","being","believe","below","beside","besides","best",
"better","between","beyond","both","brief","but","by","c","c'mon","c's","came","can","can't","cannot",
"cant","cause","causes","certain","certainly","changes","clearly","co","com","come","comes","concerning",
"consequently","consider","considering","contain","containing","contains","corresponding","could","couldn't",
"course","currently","d","definitely","described","despite","did","didn't","different","do","does","doesn't",
"doing","don't","done","down","downwards","during","e","each","edu","eg","eight","either","else","elsewhere",
"enough","entirely","especially","et","etc","even","ever","every","everybody","everyone","everything",
"everywhere","ex","exactly","example","except","f","far","few","fifth","first","five","followed","following",
"follows","for","former","formerly","forth","four","from","further","furthermore","g","get","gets","getting",
"given","gives","go","goes","going","gone","got","gotten","greetings","h","had","hadn't","happens","hardly",
"has","hasn't","have","haven't","having","he","he's","hello","help","hence","her","here","here's","hereafter",
"hereby","herein","hereupon","hers","herself","hi","him","himself","his","hither","hopefully","how","howbeit",
"however","i","i'd","i'll","i'm","i've","ie","if","ignored","immediate","in","inasmuch","inc","indeed","indicate",
"indicated","indicates","inner","insofar","instead","into","inward","is","isn't","it","it'd","it'll","it's",
"its","itself","j","just","k","keep","keeps","kept","know","knows","known","l","last","lately","later","latter"
,"latterly","least","less","lest","let","let's","like","liked","likely","little","look","looking","looks","ltd",
"m","mainly","many","may","maybe","me","mean","meanwhile","merely","might","more","moreover","most","mostly",
"much","must","my","myself","n","name","namely","nd","near","nearly","necessary","need","needs","neither",
"never","nevertheless","new","next","nine","no","nobody","non","none","noone","nor","normally","not",
"nothing","novel","now","nowhere","o","obviously","of","off","often","oh","ok","okay","old","on","once",
"one","ones","only","onto","or","other","others","otherwise","ought","our","ours","ourselves","out","outside",
"over","overall","own","p","particular","particularly","per","perhaps","placed","please","plus","possible",
"presumably","probably","provides","q","que","quite","qv","r","rather","rd","re","really","reasonably",
"regarding","regardless","regards","relatively","respectively","right","s","said","same","saw","say",
"saying","says","second","secondly","see","seeing","seem","seemed","seeming","seems","seen","self","selves",
"sensible","sent","serious","seriously","seven","several","shall","she","should","shouldn't","since","six",
"so","some","somebody","somehow","someone","something","sometime","sometimes","somewhat","somewhere","soon",
"sorry","specified","specify","specifying","still","sub","such","sup","sure","t","t's","take","taken","tell",
"tends","th","than","thank","thanks","thanx","that","that's","thats","the","their","theirs","them","themselves",
"then","thence","there","there's","thereafter","thereby","therefore","therein","theres","thereupon","these",
"they","they'd","they'll","they're","they've","think","third","this","thorough","thoroughly","those","though",
"three","through","throughout","thru","thus","to","together","too","took","toward","towards","tried","tries",
"truly","try","trying","twice","two","u","un","under","unfortunately","unless","unlikely","until","unto","up",
"upon","us","use","used","useful","uses","using","usually","uucp","v","value","various","very","via","viz",
"vs","w","want","wants","was","wasn't","way","we","we'd","we'll","we're","we've","welcome","well","went",
"were","weren't","what","what's","whatever","when","whence","whenever","where","where's","whereafter",
"whereas","whereby","wherein","whereupon","wherever","whether","which","while","whither","who","who's",
"whoever","whole","whom","whose","why","will","willing","wish","with","within","without","won't","wonder",
"would","would","wouldn't","x","y","yes","yet","you","you'd","you'll","you're","you've","your","yours",
"yourself","yourselves","z","zero"])
stopwords = set(STOPWORDS) | additional_stopwords


<h2>Idea No 1<a class="anchor-link" ></a></h2>
<p>Our first idea is  try to find tag words, after training the already occured set of tokens in the datasets df1 to df6. </p>
<p> We will construct a dataframe with the whole set of token/words in title+content and set a column with value zero if that word didn't appear as tag , or value one if that word actually appeared as a tag. </p>

<p> First we will put in a dataframe all tokens from title+content, from our train dataset from all topics (87000 posts), after cleaning it from stopwords and punctuation </p>


In [150]:
RE_PUNCTUATION = '|'.join([re.escape(x) for x in string.punctuation])
train=all_df[['title','content','tags']]
train['title'] = train['title'].str.lower().str.replace(RE_PUNCTUATION, ' ') #remove puntuation from title
train['content'] = train['content'].str.lower().str.replace(RE_PUNCTUATION, ' ')  #remove puntuation from content
train['title_content']=train['title']+train['content'] #merge the two columns of interest
train['title_content_tokens'] = train['title_content'].str.split() #split to strings
#remove stopwords
train['title_content_tokens'] = train['title_content_tokens'].apply(lambda tokens: [token for token in tokens if token not in stopwords])
# put tokens to a Dataframe after counting their frequency
tokens=train['title_content_tokens'].tolist() 
tokens = np.asarray(tokens)
tokens=[j for i in tokens for j in i]
fdist = FreqDist(tokens)
train_title_content = pd.DataFrame.from_dict(fdist, orient='index').reset_index()
#rename columns
train_title_content.rename(columns = {'index':'Token', 0:'Frequency'}, inplace=True)
#sorting
train_title_content=train_title_content[['Token','Frequency']].sort_values(by='Frequency',ascending=False)
train_title_content.head(20)

C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/st

,Token,Frequency
29743,water,20356
60457,jpg,19825
46864,visa,17096
50362,wall,11305
17101,house,11065
119890,travel,8930
26012,wire,8432
82924,switch,7925
43036,uk,7237
44876,floor,7154


<p> Now let's put unique tags from the 87000 posts in a dataframe </p>

In [151]:
train['tags_tokens'] = train['tags'].str.split()
tokens_tags=train['tags_tokens'].tolist()
tokens_tags = np.asarray(tokens_tags)
tokens_tags=[j for i in tokens_tags for j in i]
train_tags = FreqDist(tokens_tags)
train_tags = pd.DataFrame.from_dict(train_tags, orient='index').reset_index()
train_tags.rename(columns = {'index':'Token', 0:'Frequency_Tag'}, inplace=True)
train_tags=train_tags[['Token','Frequency_Tag']].sort_values(by='Frequency_Tag',ascending=False)
train_tags.shape

(4268, 2)

<p> As we see there are 4268 different tags</p> <p> Now let's see the most frequent one's </p>

In [152]:
train_tags.head(20)


,Token,Frequency_Tag
2982,electrical,4490
1892,visas,3829
1832,air-travel,2273
2056,plumbing,2223
2230,usa,2168
2628,encryption,1783
2380,wiring,1680
3589,schengen,1561
3415,uk,1522
3096,human-biology,1448


<p> Now we will merge the two dataframe on column token </p>

In [153]:
train_tokens = pd.merge(train_title_content, train_tags,how='left', on='Token')
train_tokens=train_tokens.fillna(0) #fill NaN


In [154]:
train_tokens.head(50)

,Token,Frequency,Frequency_Tag
0,water,20356,861.0
1,jpg,19825,0.0
2,visa,17096,0.0
3,wall,11305,0.0
4,house,11065,0.0
5,travel,8930,0.0
6,wire,8432,102.0
7,switch,7925,500.0
8,uk,7237,1522.0
9,floor,7154,371.0


<p> We have an indication that  many frequent words are appearing actually as tags </p><p> Let's compute the Relative Frequencies and train our model </p>

In [155]:
train_tokens['Relative_Freq']=train_tokens['Frequency']/train_tokens['Frequency'].sum()


In [156]:
train_tokens.head(10)


,Token,Frequency,Frequency_Tag,Relative_Freq
0,water,20356,861.0,0.004917
1,jpg,19825,0.0,0.004789
2,visa,17096,0.0,0.004130
3,wall,11305,0.0,0.002731
4,house,11065,0.0,0.002673
5,travel,8930,0.0,0.002157
6,wire,8432,102.0,0.002037
7,switch,7925,500.0,0.001914
8,uk,7237,1522.0,0.001748
9,floor,7154,371.0,0.001728


<p> Now we will put a dummy variable for each token, if it appeared actually as a tag </p>

In [157]:
train_tokens['tag']=''
train_tokens['tag'] = (train_tokens['Frequency_Tag'] > 0)
train_tokens['tag']=train_tokens['tag']*1

In [158]:
train_tokens.head(10)

,Token,Frequency,Frequency_Tag,Relative_Freq,tag
0,water,20356,861.0,0.004917,1
1,jpg,19825,0.0,0.004789,0
2,visa,17096,0.0,0.004130,0
3,wall,11305,0.0,0.002731,0
4,house,11065,0.0,0.002673,0
5,travel,8930,0.0,0.002157,0
6,wire,8432,102.0,0.002037,1
7,switch,7925,500.0,0.001914,1
8,uk,7237,1522.0,0.001748,1
9,floor,7154,371.0,0.001728,1


<p> Now we will train a model using <a href="https://en.wikipedia.org/wiki/Logistic_regression">Logistic Regression </a> with the token's Relative Frequencies as independent variable and the dummy 'tag' variable as dependent categorical variable. This way we will try to catch in the physics category, if the Relative Frequency of a token in the title+content, can predict if that word will be actually a tag. </p>

In [159]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

# train the model 
X=np.transpose(np.matrix(train_tokens['Relative_Freq']))
y=np.transpose(np.matrix(train_tokens['tag']))
model.fit(X, y)

C:\Users\Owner\Anaconda3\lib\site-packages\sklearn\utils\validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [160]:
# check the accuracy on the training set
model.score(X, y)

0.98237769594950031

<p> Now we will  put the physics-tokens in a dataframe, we will compute their Relative Frequency and try to predict if they will be actually a tag throught their relative frequency in the title+content. </p>

In [161]:
physics=df7
physics['title'] = physics['title'].str.lower().str.replace(RE_PUNCTUATION, ' ')
physics['content'] = physics['content'].str.lower().str.replace(RE_PUNCTUATION, ' ')
physics['title_content']=physics['title']+physics['content']
physics['title_content_tokens'] = physics['title_content'].str.split()
physics['title_content_tokens'] = physics['title_content_tokens'].apply(lambda tokens: [token for token in tokens if token not in stopwords])
tokens_new=physics['title_content_tokens'].tolist()
tokens_new = np.asarray(tokens_new)
tokens_new=[j for i in tokens_new for j in i]
fdist_new = FreqDist(tokens_new)
physics_tokens = pd.DataFrame.from_dict(fdist_new, orient='index').reset_index()
physics_tokens.rename(columns = {'index':'Token', 0:'Frequency'}, inplace=True)
physics_tokens['Relative_Freq']=physics_tokens['Frequency']/physics_tokens['Frequency'].sum()


In [162]:
physics_tokens=physics_tokens[['Token','Frequency','Relative_Freq']].sort_values(by='Relative_Freq',ascending=False)
physics_tokens.head(10)


,Token,Frequency,Relative_Freq
53127,energy,38682,0.008551
75989,field,28503,0.006301
52305,partial,24065,0.005320
26094,equation,22814,0.005043
78656,force,22507,0.004975
81522,mass,21370,0.004724
24578,left,20097,0.004442
49189,phi,18552,0.004101
100041,space,18538,0.004098
87928,physics,18377,0.004062


<p> Now we will predict if a token will be a tag, through its relative frequency</p>

In [163]:
physics_tokens_test=np.transpose(np.matrix(physics_tokens['Relative_Freq']))
predicted =model.predict(physics_tokens_test)

In [164]:
#check if token took value 1 as a tag
sum(predicted)

0

<p> So our idea didn't work, no tags where predicted </p>

<h2>Idea No 2<a class="anchor-link" ></a></h2>
<p>We will use a Python module implementation of the Rapid Automatic Keyword Extraction (RAKE) algorithm as described in: Rose, S., Engel, D., Cramer, N., & Cowley, W. (2010). Automatic Keyword Extraction from Individual Documents </p>
<p> That means will not take into account the allready existed topics. </p>
<p> RAKE module needs a list of stopwords to be given as a txt file.</p>

In [165]:
import RAKE
import operator

Rake = RAKE.Rake('SmartStoplist.txt')


In [166]:
Rake.run(str(physics['title_content'][0]))

#because of memory errors we will predict tags only for 20 first posts
#for i in range(0,len(physics['title_content']):
for i in range(0,20):
    a= Rake.run(str(physics['title_content'][i]))
    a=dict(a)
    myList = sorted(a, key=a.get, reverse=True)
    if len(myList)>=2:
        physics['tags'][i]=myList[0]+myList[1]
    else:
        physics['tags'][i]=myList[0]
    physics['tags'][i]=re.sub(r'\w*\d\w*', '', df7['tags'][i]).strip()
    physics['tags'][i]=physics['tags'][i].split()
    physics['tags'][i]= " ".join(sorted(set(physics['tags'][i]), key=physics['tags'][i].index))

#tocsv=physics[['id','tags']]
#tocsv.to_csv('example.csv', sep=',', encoding='utf-8', index=False)
physics[['id','tags']].head(20)


C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel\__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel\__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel\__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

,id,tags
0,1,actual spinning motionproperty called spin
1,2,explain string theorystring theory
2,3,lie theory representationslie groups
3,7,uncertainty principlemain problems
4,9,action integraldynamic system
5,13,term soundsound
6,15,support string theorydisprove theory
7,17,sky change colorsunrise set
8,19,particle collisions calculatedhigher energy
9,21,monte carlo methodmonte


<h2>Idea No 3<a class="anchor-link" ></a></h2>
<p><u>First</u>: We will explore the frequencies of the actual token-tags and we will find an average number of most usual token-tags in the  known topics.</p>
<p><u>Second</u>: As we find how many are actually the most important/frequent tags, we will use the LDA method with the logic that in every topic, there must be subtopics, so will find top words for every subtopic and therefore for every topic.</p>
<p><u>Third</u>: We will apply this idea to a topic as robotics to check if the predicted important words from LDA method, are actually tags, and we will calculate the differences/distance with a python's package that can be found here <a href="https://pypi.python.org/pypi/Distance/">https://pypi.python.org/pypi/Distance/</a> </p>
<p><u>Forth</u>: We will apply the method to physics to extract possible tags. </p>

<p>Let's explore some descriptive statistics in each topic </>

In [167]:
# EXPLORE biology
#remove punctuation and prepare tokens to constuct a dataframe 
df1['title'] = df1['title'].str.lower().str.replace(RE_PUNCTUATION, ' ')
df1['content'] = df1['content'].str.lower().str.replace(RE_PUNCTUATION, ' ')
df1['tags'] = df1['tags'].str.lower().str.replace(RE_PUNCTUATION, ' ')
df1['title_content']=df1['title']+df1['content']
#create Dataframe with the allready existed tags
df1['tags_tokens'] = df1['tags'].str.split()
tokens_tags=df1['tags_tokens'].tolist()
tokens_tags = np.asarray(tokens_tags)
tokens_tags=[j for i in tokens_tags for j in i]
number_of_tags=len(tokens_tags) #total number of tags
Biology_tags = FreqDist(tokens_tags)
Biology_tags = pd.DataFrame.from_dict(Biology_tags, orient='index').reset_index()
Biology_tags.rename(columns = {'index':'Tag_Name', 0:'Frequency'}, inplace=True)
Biology_tags.head(10)

,Tag_Name,Frequency
0,asexual,3
1,quantitative,25
2,identification,462
3,anecdotal,2
4,flowers,21
5,histology,40
6,untagged,6
7,matrix,4
8,lymph,4
9,energy,35


In [168]:
#explore Biology tags and find Frequencies
Biology_tags=Biology_tags[['Tag_Name','Frequency']].sort_values(by='Frequency',ascending=False)
Biology_tags['Frequency'].describe()


count     710.000000
mean       61.198592
std       214.452480
min         1.000000
25%         5.250000
50%        14.000000
75%        40.000000
max      3686.000000
Name: Frequency, dtype: float64

In [169]:
Biology_tags['Relative_Frequency']=Biology_tags['Frequency']/Biology_tags['Frequency'].sum()
NUMBER_of_different_Biology_tags=len(Biology_tags)
NUMBER_of_different_Biology_tags

710

In [170]:
# see tags above quantile 95%
Biology_tags.loc[Biology_tags['Relative_Frequency']>Biology_tags['Relative_Frequency'].quantile(0.95)]


,Tag_Name,Frequency,Relative_Frequency
234,biology,3686,0.084831
688,human,2218,0.051046
648,genetics,1942,0.044694
246,evolution,1302,0.029965
586,molecular,1272,0.029274
193,cell,1161,0.026720
348,biochemistry,984,0.022646
282,dna,977,0.022485
206,physiology,896,0.020621
229,bioinformatics,663,0.015259


<p> We set 0.001 as a limit frequency "for more important tags" and we will find the percentage of total tag-words in every topic that is covered from those words</p>

In [171]:
# tags with relative frequency above 0.001
NUMBER_of_Biology_tags_above_limit=len(Biology_tags.loc[Biology_tags['Relative_Frequency']>0.001])
Biology_tags.loc[Biology_tags['Relative_Frequency']>0.001]


,Tag_Name,Frequency,Relative_Frequency
234,biology,3686,0.084831
688,human,2218,0.051046
648,genetics,1942,0.044694
246,evolution,1302,0.029965
586,molecular,1272,0.029274
193,cell,1161,0.026720
348,biochemistry,984,0.022646
282,dna,977,0.022485
206,physiology,896,0.020621
229,bioinformatics,663,0.015259


In [172]:
NUMBER_of_biology_posts=len(df1)
NUMBER_of_biology_posts

13196

<p> We will do the same for the other topics as well</p>

In [173]:
# EXPLORE cooking
#remove punctuation 
df2['title'] = df2['title'].str.lower().str.replace(RE_PUNCTUATION, ' ')
df2['content'] = df2['content'].str.lower().str.replace(RE_PUNCTUATION, ' ')
df2['tags'] = df2['tags'].str.lower().str.replace(RE_PUNCTUATION, ' ')
df2['title_content']=df2['title']+df2['content']
#create Dataframe with the allready existed tags
df2['tags_tokens'] = df2['tags'].str.split()
tokens_tags=df2['tags_tokens'].tolist()
tokens_tags = np.asarray(tokens_tags)
tokens_tags=[j for i in tokens_tags for j in i]
number_of_tags=len(tokens_tags) #total number of tags
Cooking_tags = FreqDist(tokens_tags)
Cooking_tags = pd.DataFrame.from_dict(Cooking_tags, orient='index').reset_index()
Cooking_tags.rename(columns = {'index':'Tag_Name', 0:'Frequency'}, inplace=True)
#explore tags
Cooking_tags=Cooking_tags[['Tag_Name','Frequency']].sort_values(by='Frequency',ascending=False)
Cooking_tags['Frequency'].describe()
Cooking_tags['Relative_Frequency']=Cooking_tags['Frequency']/Cooking_tags['Frequency'].sum()
NUMBER_of_Cooking_tags_above_limit=len(Cooking_tags.loc[Cooking_tags['Relative_Frequency']>0.001])
NUMBER_of_Cooking_posts=len(df2)
NUMBER_of_different_Cooking_tags=len(Cooking_tags)

In [174]:
# EXPLORE crypto
#remove punctuation 
df3['title'] = df3['title'].str.lower().str.replace(RE_PUNCTUATION, ' ')
df3['content'] = df3['content'].str.lower().str.replace(RE_PUNCTUATION, ' ')
df3['tags'] = df3['tags'].str.lower().str.replace(RE_PUNCTUATION, ' ')
df3['title_content']=df3['title']+df3['content']
#create Dataframe with the allready existed tags
df3['tags_tokens'] = df3['tags'].str.split()
tokens_tags=df3['tags_tokens'].tolist()
tokens_tags = np.asarray(tokens_tags)
tokens_tags=[j for i in tokens_tags for j in i]
number_of_tags=len(tokens_tags) #total number of tags
Crypto_tags = FreqDist(tokens_tags)
Crypto_tags = pd.DataFrame.from_dict(Crypto_tags, orient='index').reset_index()
Crypto_tags.rename(columns = {'index':'Tag_Name', 0:'Frequency'}, inplace=True)
#explore tags
Crypto_tags=Crypto_tags[['Tag_Name','Frequency']].sort_values(by='Frequency',ascending=False)
Crypto_tags['Frequency'].describe()
Crypto_tags['Relative_Frequency']=Crypto_tags['Frequency']/Crypto_tags['Frequency'].sum()
NUMBER_of_Crypto_tags_above_limit=len(Crypto_tags.loc[Crypto_tags['Relative_Frequency']>0.001])
NUMBER_of_Crypto_posts=len(df3)
NUMBER_of_different_Crypto_tags=len(Crypto_tags)

In [175]:
# EXPLORE diy
#remove punctuation 
df4['title'] = df4['title'].str.lower().str.replace(RE_PUNCTUATION, ' ')
df4['content'] = df4['content'].str.lower().str.replace(RE_PUNCTUATION, ' ')
df4['tags'] = df4['tags'].str.lower().str.replace(RE_PUNCTUATION, ' ')
df4['title_content']=df4['title']+df4['content']
#create Dataframe with the allready existed tags
df4['tags_tokens'] = df4['tags'].str.split()
tokens_tags=df4['tags_tokens'].tolist()
tokens_tags = np.asarray(tokens_tags)
tokens_tags=[j for i in tokens_tags for j in i]
number_of_tags=len(tokens_tags) #total number of tags
Diy_tags = FreqDist(tokens_tags)
Diy_tags = pd.DataFrame.from_dict(Diy_tags, orient='index').reset_index()
Diy_tags.rename(columns = {'index':'Tag_Name', 0:'Frequency'}, inplace=True)
#explore tags
Diy_tags=Diy_tags[['Tag_Name','Frequency']].sort_values(by='Frequency',ascending=False)
Diy_tags['Frequency'].describe()
Diy_tags['Relative_Frequency']=Diy_tags['Frequency']/Diy_tags['Frequency'].sum()
NUMBER_of_Diy_tags_above_limit=len(Diy_tags.loc[Diy_tags['Relative_Frequency']>0.001])
NUMBER_of_Diy_posts=len(df4)
NUMBER_of_different_Diy_tags=len(Diy_tags)

In [176]:
# EXPLORE ROBOTICS
#remove punctuation 
df5['title'] = df5['title'].str.lower().str.replace(RE_PUNCTUATION, ' ')
df5['content'] = df5['content'].str.lower().str.replace(RE_PUNCTUATION, ' ')
df5['tags'] = df5['tags'].str.lower().str.replace(RE_PUNCTUATION, ' ')
df5['title_content']=df5['title']+df5['content']
#create Dataframe with the allready existed tags
df5['tags_tokens'] = df5['tags'].str.split()
tokens_tags=df5['tags_tokens'].tolist()
tokens_tags = np.asarray(tokens_tags)
tokens_tags=[j for i in tokens_tags for j in i]
number_of_tags=len(tokens_tags) #total number of tags
Robotics_tags = FreqDist(tokens_tags)
Robotics_tags = pd.DataFrame.from_dict(Robotics_tags, orient='index').reset_index()
Robotics_tags.rename(columns = {'index':'Tag_Name', 0:'Frequency'}, inplace=True)
#explore tags
Robotics_tags=Robotics_tags[['Tag_Name','Frequency']].sort_values(by='Frequency',ascending=False)
Robotics_tags['Frequency'].describe()
Robotics_tags['Relative_Frequency']=Robotics_tags['Frequency']/Robotics_tags['Frequency'].sum()
NUMBER_of_Robotics_tags_above_limit=len(Robotics_tags.loc[Robotics_tags['Relative_Frequency']>0.001])
NUMBER_of_robotic_posts=len(df5)
NUMBER_of_different_robotic_tags=len(Robotics_tags)

In [177]:
# EXPLORE Travel
#remove punctuation 
df6['title'] = df6['title'].str.lower().str.replace(RE_PUNCTUATION, ' ')
df6['content'] = df6['content'].str.lower().str.replace(RE_PUNCTUATION, ' ')
df6['tags'] = df6['tags'].str.lower().str.replace(RE_PUNCTUATION, ' ')
df6['title_content']=df6['title']+df6['content']
#create Dataframe with the allready existed tags
df6['tags_tokens'] = df6['tags'].str.split()
tokens_tags=df6['tags_tokens'].tolist()
tokens_tags = np.asarray(tokens_tags)
tokens_tags=[j for i in tokens_tags for j in i]
number_of_tags=len(tokens_tags) #total number of tags
Travel_tags = FreqDist(tokens_tags)
Travel_tags = pd.DataFrame.from_dict(Travel_tags, orient='index').reset_index()
Travel_tags.rename(columns = {'index':'Tag_Name', 0:'Frequency'}, inplace=True)
#explore tags
Travel_tags=Travel_tags[['Tag_Name','Frequency']].sort_values(by='Frequency',ascending=False)
Travel_tags['Relative_Frequency']=Travel_tags['Frequency']/Travel_tags['Frequency'].sum()
Travel_tags.loc[Travel_tags['Relative_Frequency']>Travel_tags['Relative_Frequency'].quantile(0.95)]
Travel_tags['Frequency'].quantile(0.95)
Travel_tags.loc[Travel_tags['Relative_Frequency']>0.001]
NUMBER_of_Travel_posts=len(df6)
NUMBER_of_different_Travel_tags=len(Travel_tags)
NUMBER_of_Travel_tags_above_limit=len(Travel_tags.loc[Travel_tags['Relative_Frequency']>0.001])

<p> Now we will find the percentages tag-words that are covered from the important tags with relative frequency above 0.001</p>

In [178]:
percentage_biology=Biology_tags['Frequency'][0:NUMBER_of_Biology_tags_above_limit].sum()/Biology_tags['Frequency'].sum()
percentage_cooking=Cooking_tags['Frequency'][0:NUMBER_of_Cooking_tags_above_limit].sum()/Cooking_tags['Frequency'].sum()
percentage_robotics=Robotics_tags['Frequency'][0:NUMBER_of_Robotics_tags_above_limit].sum()/Robotics_tags['Frequency'].sum()
percentage_Crypto=Crypto_tags['Frequency'][0:NUMBER_of_Crypto_tags_above_limit].sum()/Crypto_tags['Frequency'].sum()
percentage_Diy=Diy_tags['Frequency'][0:NUMBER_of_Diy_tags_above_limit].sum()/Diy_tags['Frequency'].sum()
percentage_travel=Travel_tags['Frequency'][0:NUMBER_of_Travel_tags_above_limit].sum()/Travel_tags['Frequency'].sum()

In [179]:
 d = {'Number_posts' : pd.Series([NUMBER_of_biology_posts, NUMBER_of_Cooking_posts,NUMBER_of_Crypto_posts,NUMBER_of_Diy_posts,NUMBER_of_robotic_posts,NUMBER_of_Travel_posts],index=['Biology', 'Cooking', 'Crypto', 'Diy','Robotics','Travel']),
     'Number_of_Different_tags' : pd.Series([NUMBER_of_different_Biology_tags, NUMBER_of_different_Cooking_tags, NUMBER_of_different_Crypto_tags, NUMBER_of_different_Diy_tags,NUMBER_of_different_robotic_tags,NUMBER_of_different_Travel_tags], index=['Biology', 'Cooking', 'Crypto', 'Diy','Robotics','Travel']),
     'Important_Tags_above_limit': pd.Series([NUMBER_of_Biology_tags_above_limit, NUMBER_of_Cooking_tags_above_limit, NUMBER_of_Crypto_tags_above_limit, NUMBER_of_Diy_tags_above_limit,NUMBER_of_Robotics_tags_above_limit,NUMBER_of_Travel_tags_above_limit], index=['Biology', 'Cooking', 'Crypto', 'Diy','Robotics','Travel']),
    'Tag_Percentage_Covered_by_Important_Tags': pd.Series([percentage_biology, percentage_cooking, percentage_Crypto, percentage_Diy,percentage_robotics,percentage_travel], index=['Biology', 'Cooking', 'Crypto', 'Diy','Robotics','Travel'])}

Tag_stat = pd.DataFrame(d)
Tag_stat

,Important_Tags_above_limit,Number_of_Different_tags,Number_posts,Tag_Percentage_Covered_by_Important_Tags
Biology,165,710,13196,0.838715
Cooking,207,748,15404,0.815615
Crypto,160,472,10432,0.903378
Diy,220,742,25918,0.847878
Robotics,151,266,2771,0.947656
Travel,188,1723,19279,0.750286


In [180]:
Tag_stat['Number_posts'].sum()

87000

In [181]:
Tag_stat['Number_of_Different_tags'].sum()

4661

In [182]:
Tag_stat['Tag_Percentage_Covered_by_Important_Tags'].mean()

0.8505880965025491

In [183]:
Tag_stat['Important_Tags_above_limit'].mean()

181.83333333333334

<p>As we see above the most important tags are covering a 85% of the total token-tag-words, with an average number of 181 per category</p>
<p>Let's check now our idea by tring it to robotics</p>
<p></p>
<p>We will use LDA to robotics</p>

In [184]:
robotics=df5 
robotics['title'] = robotics['title'].str.lower().str.replace(RE_PUNCTUATION, ' ')
robotics['content'] = robotics['content'].str.lower().str.replace(RE_PUNCTUATION, ' ')
robotics['title_content']=robotics['title']+robotics['content']
robotics=robotics[['id','title_content']]
robotics.head(3)

,id,title_content
0,1,what is the right approach to write the spin c...
1,2,how can i modify a low cost hobby servo to run...
2,3,what useful gaits exist for a six legged robot...


<p> Now we suppose 10 different subtopics for robotics with 15 top words in each in order to make a bag of important words in robotics </p>

In [185]:
n_topics = 10
n_top_words = 15

def top_words(model, feature_names, n_top_words):
    tags = []    
    for topic_idx, topic in enumerate(model.components_):
        tags.append({'Topic': topic_idx, 'Tags': " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])})        
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    tags=pd.DataFrame(tags)
    return tags
    print()
    
        

tf_vectorizer = CountVectorizer(ngram_range=(1, 1), min_df=10,stop_words=stopwords)
tf = tf_vectorizer.fit_transform(robotics['title_content'])

lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

lda.fit(tf)
tf_feature_names = tf_vectorizer.get_feature_names()
robotic_top=top_words(lda, tf_feature_names, n_top_words)

Topic #0:
motor motors speed torque dc control quadcopter power jpg current esc stepper pwm gear controller
Topic #1:
0x00 serial data void double acc output return write sensor pin arduino input print gpio
Topic #2:
robot arduino robotics control project raspberry board sensors usb software working sensor camera build program
Topic #3:
robot create space obstacle obstacles roomba irobot pdf planning 00 surface path goal files mobile
Topic #4:
state filter slam map kalman pose function matrix algorithm measurement odometry model robot vector ekf
Topic #5:
pid error ros controller output loop youtube watch control kp quadcopter gain arm kd package
Topic #6:
servo arm robot robots robotic servos stereo vision design systems cameras weight cost low torque
Topic #7:
robot position sensor angle distance velocity pitch data roll yaw imu axis rate sensors path
Topic #8:
battery batteries wire power air lipo design gps charge charger area pole antenna supply circuit
Topic #9:
theta float joint

In [186]:
robotic_top

,Tags,Topic
0,motor motors speed torque dc control quadcopte...,0
1,0x00 serial data void double acc output return...,1
2,robot arduino robotics control project raspber...,2
3,robot create space obstacle obstacles roomba i...,3
4,state filter slam map kalman pose function mat...,4
5,pid error ros controller output loop youtube w...,5
6,servo arm robot robots robotic servos stereo v...,6
7,robot position sensor angle distance velocity ...,7
8,battery batteries wire power air lipo design g...,8
9,theta float joint matrix axis dot cos position...,9


In [187]:
#put tags in a list
robotic_top['Tags']=robotic_top['Tags'].str.split()
robotics_tags=[]
for i in range(0,len(robotic_top)):
    robotics_tags+=robotic_top['Tags'][i]

In [188]:
#keep unique
robotics_tags = list(set(robotics_tags))


In [189]:
#see how many tags are unique
len(robotics_tags)

125

<p> We will search each robotic post (title+content) if it contains the word-tokens predicted from LDA method as most important in robotics subtopics. For memory error reasons we will do it ontly for the first 30 posts</p>

In [190]:
robotics['title_content']=robotics['title_content'].str.split()
robotics['title_content'] = robotics['title_content'].apply(lambda tokens: [token for token in tokens if token not in stopwords])
robotics['tags']=''

for i in range(0,30):
    for j in robotics_tags:
        if j in robotics['title_content'][i]:
            robotics['tags'][i]+="".join(str(j)+" ")

C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

In [191]:
robotics.head(30)


,id,title_content,tags
0,1,"[approach, write, spin, controller, soccer, ro...",stepper software pwm angle goal write void pid...
1,2,"[modify, low, cost, hobby, servo, freely, hobb...",low software arduino angle state cost power se...
2,3,"[gaits, exist, legged, robot, pros, cons, oric...",robot
3,4,"[microcontrollers, socs, robotics, project, st...",arduino raspberry project build robotics systems
4,5,"[nearest, neighbor, data, structure, euclidean...",data algorithm arm working space kd serial robot
5,6,"[robotics, software, platforms, operating, sys...",software data project design robotics robotic ...
6,11,"[software, design, pcb, robotics, field, softw...",software design robots robotics
7,18,"[methods, tuning, process, noise, kalman, filt...",kalman filter error
8,19,"[keyboard, control, map, scalar, based, moveme...",working controller control map
9,20,"[ideas, shooting, ball, soccer, robot, option,...",robot


<p>Explore the distances (levenshtein,sorensen and jaccard methods) between tags predicted and the actual tags to see how much difference is there 

In [192]:
import distance

robotics['distance']=''
for i in range(0,30):
    robotics['distance'][i]=distance.nlevenshtein(robotics['tags'][i], df5['tags'][i], method=1)  # shortest alignment
robotics['distance'][0:30].mean()

C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Owner\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2881: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


0.79076853110608591

In [193]:
robotics['distance2']=''
for i in range(0,30):
    robotics['distance2'][i]=distance.sorensen(robotics['tags'][i], df5['tags'][i])  # shortest alignment
robotics['distance2'][0:30].mean()

C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\Owner\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2881: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user

0.30996311792366771

In [194]:
robotics['distance3']=''
for i in range(0,30):
    robotics['distance3'][i]=distance.jaccard(robotics['tags'][i], df5['tags'][i])  # shortest alignment
robotics['distance3'][0:30].mean()

C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\Owner\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2881: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user

0.44960393278891736

<p> So there is an indication that the actual tags are partly predicted with our method.Sorensen's method shows the closest distance.</p> 
<p> We will continue by finding  250 important words (Above mean fo the rest of the topics that is 181) with LDA method, assuming there are 10 subtopics in physics with 25 top words each. </p>

In [195]:
n_topics = 10
n_top_words = 25
tf_vectorizer = CountVectorizer(ngram_range=(1, 1), min_df=10,stop_words=stopwords)
tf = tf_vectorizer.fit_transform(df7['title_content'])
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

lda.fit(tf)
tf_feature_names = tf_vectorizer.get_feature_names()
physic_top=top_words(lda, tf_feature_names, n_top_words)

Topic #0:
space bar frame theory vector symmetry tensor gauge field string metric group spacetime relativity lorentz transformation reference general action lagrangian dimensions spin invariant fields scalar
Topic #1:
speed earth universe space gravity matter observer sun gravitational big star orbit distance event dark moving travel planet horizon wikipedia faster object mass objects relative
Topic #2:
energy water temperature pressure heat air gas kinetic volume fluid liquid potential density thermal constant surface molecules increase entropy equilibrium law process ideal tube rate
Topic #3:
electron energy photon frequency spin electrons photons atom sound momentum decay scattering wavelength hydrogen spectrum proton amplitude band level neutron nucleus atomic emitted angular radiation
Topic #4:
physics quantum theory function mechanics physical model wikipedia classical phase equation book point order principle equations functions real mathematical position answer space hamiltonia

In [196]:
physic_top

,Tags,Topic
0,space bar frame theory vector symmetry tensor ...,0
1,speed earth universe space gravity matter obse...,1
2,energy water temperature pressure heat air gas...,2
3,electron energy photon frequency spin electron...,3
4,physics quantum theory function mechanics phys...,4
5,current jpg flow wire power voltage circuit be...,5
6,field particle particles magnetic wave charge ...,6
7,partial left equation phi delta mathbf theta a...,7
8,force velocity mass electric direction point o...,8
9,rangle state langle dagger states quantum oper...,9


In [197]:
physic_top2=physic_top
physic_top2['Tags']=physic_top2['Tags'].str.split()
physics_tags=[]
for i in range(0,len(physic_top2)):
    physics_tags+=physic_top2['Tags'][i]
physics_tags = list(set(physics_tags)) #keep unique
physics_tags #see possible tags

['material',
 'lagrangian',
 'lens',
 'group',
 'ln',
 'matrix',
 'molecules',
 'particle',
 'reference',
 'positive',
 'energy',
 'relative',
 'model',
 'voltage',
 'forces',
 'wave',
 'function',
 'photons',
 'beta',
 'string',
 'quantum',
 'sun',
 'faster',
 'volume',
 'level',
 'constant',
 'relativity',
 'big',
 'object',
 'box',
 'gravity',
 'universe',
 'partial',
 'objects',
 'delta',
 'simple',
 'radiation',
 'point',
 'nucleus',
 'sqrt',
 'potential',
 'jpg',
 'dark',
 'water',
 'ball',
 'flow',
 'cos',
 'magnet',
 'phase',
 'basis',
 'text',
 'entropy',
 'vector',
 'kinetic',
 'charges',
 'sound',
 'momentum',
 'distance',
 'friction',
 'theta',
 'probability',
 'law',
 'magnetic',
 'gauge',
 'horizon',
 'rate',
 'position',
 'metal',
 'cdot',
 'direction',
 'operator',
 'angle',
 'vacuum',
 'length',
 'tube',
 'hamiltonian',
 'resistance',
 'intensity',
 'photon',
 'gas',
 'state',
 'travel',
 'frame',
 'scalar',
 'answer',
 'sigma',
 'earth',
 'observer',
 'moving',
 'temp

In [198]:
len(physics_tags)  #see unique

220

In [199]:
df7 = pd.read_csv('test.csv')
df7['title_content']=df7['title']+df7['title']
df7['tags']=""
df7['title_content'] = df7['title_content'].str.lower().str.replace(RE_PUNCTUATION, ' ')
df_test=df7[['id','title_content','tags']]
df_test['title_content_tokens'] = df_test['title_content'].str.split() #split to strings
#remove stopwords
df_test['title_content_tokens'] = df_test['title_content_tokens'].apply(lambda tokens: [token for token in tokens if token not in stopwords])


C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [200]:
df_test.head(20)

,id,title_content,tags,title_content_tokens
0,1,what is spin as it relates to subatomic partic...,,"[spin, relates, subatomic, particles, spin, re..."
1,2,what is your simplest explanation of the strin...,,"[simplest, explanation, string, theory, simple..."
2,3,lie theory representations and particle physi...,,"[lie, theory, representations, particle, physi..."
3,7,will determinism be ever possible will determi...,,"[determinism, determinism]"
4,9,hamilton s principlehamilton s principle,,"[hamilton, principlehamilton, principle]"
5,13,what is sound and how is it produced what is s...,,"[sound, produced, sound, produced]"
6,15,what experiment would disprove string theory w...,,"[experiment, disprove, string, theory, experim..."
7,17,why does the sky change color why the sky is ...,,"[sky, color, sky, blue, sunrise, night, sky, c..."
8,19,how s the energy of particle collisions calcul...,,"[energy, particle, collisions, calculated, ene..."
9,21,monte carlo usemonte carlo use,,"[monte, carlo, usemonte, carlo]"


In [201]:
#for i in range(0,len(df7)):
for i in range(0,200):
    for j in physics_tags:
        if j in df_test['title_content_tokens'][i]:
            df_test['tags'][i]+="".join(str(j)+" ")

C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Owner\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2881: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [202]:
df_test.head(200)

,id,title_content,tags,title_content_tokens
0,1,what is spin as it relates to subatomic partic...,spin particles,"[spin, relates, subatomic, particles, spin, re..."
1,2,what is your simplest explanation of the strin...,string theory,"[simplest, explanation, string, theory, simple..."
2,3,lie theory representations and particle physi...,particle physics theory,"[lie, theory, representations, particle, physi..."
3,7,will determinism be ever possible will determi...,,"[determinism, determinism]"
4,9,hamilton s principlehamilton s principle,principle,"[hamilton, principlehamilton, principle]"
5,13,what is sound and how is it produced what is s...,sound,"[sound, produced, sound, produced]"
6,15,what experiment would disprove string theory w...,string experiment theory,"[experiment, disprove, string, theory, experim..."
7,17,why does the sky change color why the sky is ...,,"[sky, color, sky, blue, sunrise, night, sky, c..."
8,19,how s the energy of particle collisions calcul...,particle energy,"[energy, particle, collisions, calculated, ene..."
9,21,monte carlo usemonte carlo use,,"[monte, carlo, usemonte, carlo]"


In [203]:
df_test[['id','tags']].head(200)
#tocsv2=df_test[['id','tags']]
#tocsv2.to_csv('example2.csv', sep=',', encoding='utf-8', index=False)

,id,tags
0,1,spin particles
1,2,string theory
2,3,particle physics theory
3,7,
4,9,principle
5,13,sound
6,15,string experiment theory
7,17,
8,19,particle energy
9,21,
